# Z3 Examples - SMT Solving with Python

This notebook contains interactive examples for learning SMT solving with Z3.


In [ ]:
pip install z3-solver

Note: you may need to restart the kernel to use updated packages.


## Example 1: Basic Arithmetic


In [ ]:
from z3 import *

# Create integer variables
x = Int('x')
y = Int('y')

# Create a solver
s = Solver()

# Add constraints
s.add(x + y == 10)
s.add(x > 3)

# Check satisfiability
if s.check() == sat:
    model = s.model()
    print(f"x = {model[x]}, y = {model[y]}")
else:
    print("Unsatisfiable")


x = 4, y = 6


## Example 2: Array Bounds Checking


In [3]:
from z3 import *

def verify_array_access(array_size, user_input):
    """Verify that array access arr[index] is safe"""
    i = Int('i')
    s = Solver()

    # User input constraint
    s.add(i == user_input)

    # Check if bounds can be violated
    s.add(Not(And(0 <= i, i < array_size)))

    if s.check() == unsat:
        print(f"Array access is safe! (index {user_input} in array of size {array_size})")
    else:
        print(f"Array access may be unsafe! (index {user_input} in array of size {array_size})")
        print("Counterexample:", s.model())

# Test cases
verify_array_access(10, 5)   # Safe
verify_array_access(10, 15)  # Unsafe


Array access is safe! (index 5 in array of size 10)
Array access may be unsafe! (index 15 in array of size 10)
Counterexample: [i = 15]


## Example 3: Theory Combination (Arithmetic + Arrays)


In [4]:
from z3 import *

# Mix arithmetic and arrays in one formula
x = Int('x')
A = Array('A', IntSort(), IntSort())
B = Array('B', IntSort(), IntSort())

s = Solver()
s.add(x > 0)                    # Linear arithmetic theory
s.add(A[x] == B[x+1])           # Array theory
s.add(A[x] + x == 10)   # Combined: arrays + arithmetic

print("Checking satisfiability...")
if s.check() == sat:
    model = s.model()
    print(f"x = {model[x]}")
    print("SAT - formula is satisfiable!")
else:
    print("UNSAT - formula is unsatisfiable!")

Checking satisfiability...
x = 1
SAT - formula is satisfiable!


## Example 4: Farmer's Animals


In [5]:
from z3 import *

def solve_farmer_problem():
    """Solve: A farmer has chickens and rabbits.
    He counts 35 heads and 94 legs.
    How many chickens and rabbits does he have?"""

    c = Int('chickens')
    r = Int('rabbits')

    s = Solver()

    # Constraints
    s.add(c + r == 35)        # Total heads
    s.add(2*c + 4*r == 94)    # Total legs (chickens have 2, rabbits have 4)
    s.add(c >= 0)             # Non-negative
    s.add(r >= 0)             # Non-negative

    if s.check() == sat:
        model = s.model()
        chickens = model[c]
        rabbits = model[r]
        print(f"Solution: {chickens} chickens, {rabbits} rabbits")
        print(f"Verification: {chickens + rabbits} heads, {2*chickens + 4*rabbits} legs")
    else:
        print("No solution found")

solve_farmer_problem()


Solution: 23 chickens, 12 rabbits
Verification: 23 + 12 heads, 2*23 + 4*12 legs


## Example 5: Finding Multiple Solutions

Sometimes we want to find all possible solutions to a problem, not just one. Z3 only finds one solution at a time, so we need to use a loop to enumerate all solutions.

In [6]:
from z3 import *

def find_all_solutions():
    """Find all solutions to x + y = 10 with x ≥ 0, y ≥ 0"""

    x = Int('x')
    y = Int('y')

    s = Solver()

    # Constraints: x + y = 10, x ≥ 0, y ≥ 0
    s.add(x + y == 10)
    s.add(x >= 0)
    s.add(y >= 0)

    solutions = []

    print("Finding all solutions to: x + y = 10, x ≥ 0, y ≥ 0")
    print("-" * 50)

    while s.check() == sat:
        model = s.model()
        x_val = model[x]
        y_val = model[y]
        solutions.append((x_val, y_val))

        print(f"Solution {len(solutions)}: x = {x_val}, y = {y_val}")

        # CRITICAL: Exclude this specific solution to avoid infinite loop
        # We must exclude the complete model, not just individual variables
        s.add(Not(And(x == x_val, y == y_val)))

        # Safety check (optional, for demo purposes)
        if len(solutions) >= 10:
            print("... (stopping after 10 solutions for demo)")
            break

    print(f"\nTotal solutions found: {len(solutions)}")
    return solutions

# Run the example
solutions = find_all_solutions()

Finding all solutions to: x + y = 10, x ≥ 0, y ≥ 0
--------------------------------------------------
Solution 1: x = 10, y = 0
Solution 2: x = 0, y = 10
Solution 3: x = 1, y = 9
Solution 4: x = 2, y = 8
Solution 5: x = 3, y = 7
Solution 6: x = 4, y = 6
Solution 7: x = 5, y = 5
Solution 8: x = 6, y = 4
Solution 9: x = 7, y = 3
Solution 10: x = 8, y = 2
... (stopping after 10 solutions for demo)

Total solutions found: 10


Note the line
```python
s.add(Not(And(x == x_val, y == y_val)))  
```
which makes sure that next time round the while loop a new solution is found.